In [ ]:
import pandas



descriptors_df = pandas.read_csv('acceptor_descriptors.csv', index_col=[0, 1])   
operation_df = pandas.read_csv('00.operations.csv') 
operation_dict = dict(zip(operation_df['descriptor'], operation_df['operation'])) 
acceptor_df = pandas.read_csv('acceptor.csv')   


descriptor_column_array = descriptors_df.columns[1:]


group_rc = pandas.DataFrame(columns = descriptor_column_array)
group_rl = pandas.DataFrame(columns = descriptor_column_array)
group_rp = pandas.DataFrame(columns = descriptor_column_array)
group_pb = pandas.DataFrame(columns = descriptor_column_array)
group_sc = pandas.DataFrame(columns = descriptor_column_array)


result_sheet_column_names_list = ['Acceptor']


result_sheet_column_names_list += ['RC_' + descriptor for descriptor in descriptor_column_array.to_list()]
result_sheet_column_names_list += ['PB_' + descriptor for descriptor in descriptor_column_array.to_list()]
result_sheet_column_names_list += ['SC_' + descriptor for descriptor in descriptor_column_array.to_list()]

result_sheet_column_names_list.append('sym')  


generated_df = pandas.DataFrame(columns=pandas.Index(result_sheet_column_names_list))


group_rc.loc[0] = [0] * len(descriptor_column_array)
group_rl.loc[0] = [0] * len(descriptor_column_array)
group_rp.loc[0] = [0] * len(descriptor_column_array)
group_pb.loc[0] = [0] * len(descriptor_column_array)
group_sc.loc[0] = [0] * len(descriptor_column_array)


for idx, row in descriptors_df.iterrows():
    if row.name[0] == 'RC':
        group_rc.loc[len(group_rc)] = row[1:].values
    elif row.name[0] == 'RL':
        group_rl.loc[len(group_rl)] = row[1:].values
    elif row.name[0] == 'RP':
        group_rp.loc[len(group_rp)] = row[1:].values
    elif row.name[0] == 'PB':
        group_pb.loc[len(group_pb)] = row[1:].values
    elif row.name[0] == 'SC':
        group_sc.loc[len(group_sc)] = row[1:].values


for idx, row in acceptor_df.iterrows():
    if pandas.isna(row['acceptor']):
        break
    
    
    new_row = [row['acceptor']]

    
    rc_descript_value = group_rc.loc[row['RC']].copy()
    rl_descript_value = group_rl.loc[row['RL']].copy()
    rp_descript_value = group_rp.loc[row['RP']].copy()
    pb_descript_value = group_pb.loc[row['PB']].copy()
    sc_descript_value = group_sc.loc[row['SC']].copy()

    
    for column_key in descriptor_column_array:
        if operation_dict[column_key] == 'max':
            rc_descript_value[column_key] = max(rc_descript_value[column_key], rl_descript_value[column_key], rp_descript_value[column_key])

        elif operation_dict[column_key] == 'min':
            rc_descript_value[column_key] = min(rc_descript_value[column_key], rl_descript_value[column_key], rp_descript_value[column_key])

        elif operation_dict[column_key] == 'avg':
            rc_descript_value[column_key] = (rc_descript_value[column_key] + rl_descript_value[column_key] + rp_descript_value[column_key]) / 3

        elif operation_dict[column_key] == 'sum':
            rc_descript_value[column_key] = rc_descript_value[column_key] + rl_descript_value[column_key] + rp_descript_value[column_key]

    new_row += rc_descript_value.to_list() 
    new_row += pb_descript_value.to_list()
    new_row += sc_descript_value.to_list()
    new_row.append(row['sym'])

    
    generated_df.loc[len(generated_df)] = new_row

generated_df = generated_df.loc[:, (generated_df != 0).any(axis=0)]
generated_df.to_csv("acceptor_result.csv", index = False)

